In [1]:
pip install transformers datasets evaluate rouge_score

  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=d84ba550ed7075970c93db7448452520d8ba038220c20813d382fb3d6659bb05
  Stored in directory: c:\users\manish_bhoge\appdata\local\pip\cache\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
  Attempting uninstall: datasets
    Found existing installation: datasets 1.18.3
    Uninstalling datasets-1.18.3:
      Successfully uninstalled datasets-1.18.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Manish_Bhoge\.cache\huggingface\token
Login successful


In [4]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to C:/Users/Manish_Bhoge/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.
